In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import fooof
print(fooof.__version__)

1.0.1-dev


In [2]:
from pathlib import Path, PurePath
from glob import glob
import scipy.io  as sio
import numpy as np
import re
import xarray as xr
import pandas as pd


pandas_list = []
folder ='../data/lemon_rsEEG_dataset'
files_EC = [Path(p) for p in glob(f"{folder}/*/*EC.mat")]

files_short = files_EC

data_arrays_128freq = []
data_arrays_52freq = []
sub_128freq = []
sub_52freq= []

for path_mat in files_short:
    sub = re.findall("sub-[0-9]{6}", str(path_mat))
    mat_content = sio.loadmat(path_mat)
    roi_names=  [l.flatten()[0] for l in mat_content["RowNames"].flatten()] #flatten
    freqs = np.ndarray.squeeze(mat_content["Freqs"])
    tf = mat_content["TF"]
    da = xr.DataArray(tf, dims=['roi_names', 'sub','freqs',], coords={'roi_names': roi_names, 'sub': sub,  'freqs': freqs})
    if freqs.size >  51:
        data_arrays_128freq.append(da)
        sub_128freq.append(sub)
    else:
        data_arrays_52freq.append(da)
        sub_52freq.append(sub)

psd_concat_freq128 = xr.concat(data_arrays_128freq , dim='sub', coords='all', join='override')
psd_concat_freq128.to_netcdf("data/freq128.nc")
display(f"psd_concat_freq128 is {len(sub_128freq)} elements")


psd_concat_freq52 = xr.concat(data_arrays_52freq  , dim='sub', coords='all', join='override')
psd_concat_freq52.to_netcdf("data/freq52.nc")
display(f"psd_concat_freq52 is {len(sub_52freq)} elements")




'psd_concat_freq128 is 199 elements'

'psd_concat_freq52 is 2 elements'

In [4]:
psd_concat_freq128

<xarray.DataArray (roi_names: 68, sub: 199, freqs: 126)>
array([[[4.89888099e-19, 1.72066053e-18, 2.47387808e-18, ...,
         1.66994127e-23, 1.61905183e-23, 1.68383258e-23],
        [3.45463142e-19, 1.09094906e-18, 1.56567054e-18, ...,
         2.22436615e-23, 2.21709927e-23, 2.20611712e-23],
        [6.46858652e-19, 2.13337416e-18, 2.81250888e-18, ...,
         1.35917510e-23, 1.35515552e-23, 1.35611767e-23],
        ...,
        [3.15362243e-19, 1.00457290e-18, 1.44664494e-18, ...,
         1.06296325e-23, 1.08019148e-23, 1.06044826e-23],
        [2.90448986e-18, 9.29204998e-18, 1.18850598e-17, ...,
         8.75516352e-23, 8.76867712e-23, 8.74647658e-23],
        [2.78961336e-18, 9.13730509e-18, 1.45938952e-17, ...,
         8.74064619e-23, 8.73606475e-23, 8.73282442e-23]],

       [[3.55104505e-19, 1.22098234e-18, 1.73427504e-18, ...,
         1.47035621e-23, 1.45696721e-23, 1.46188951e-23],
        [4.10891388e-19, 1.31674717e-18, 1.76832060e-18, ...,
         2.14735402e-23, 2.21853996e-23, 2.14310042e-23],
        [7.14778883e-19, 2.31733865e-18, 3.12842732e-18, ...,
         2.79278826e-23, 2.78673172e-23, 2.78632141e-23],
...
        [1.18082021e-19, 4.01739318e-19, 4.81079949e-19, ...,
         2.54050754e-24, 2.60343891e-24, 2.53133013e-24],
        [1.23018660e-18, 4.05741626e-18, 5.06336523e-18, ...,
         2.80352086e-23, 2.80723553e-23, 2.80094438e-23],
        [7.59507278e-19, 2.47583098e-18, 3.39054873e-18, ...,
         1.61625570e-23, 1.61804443e-23, 1.61812245e-23]],

       [[1.01623470e-19, 3.44991523e-19, 5.38072054e-19, ...,
         3.96342369e-24, 3.93790670e-24, 3.92436072e-24],
        [1.44070928e-19, 4.81482624e-19, 6.51830897e-19, ...,
         5.99404630e-24, 6.09708219e-24, 5.99192334e-24],
        [2.22572101e-19, 7.53124793e-19, 1.01761689e-18, ...,
         7.47521039e-24, 7.45533632e-24, 7.45074200e-24],
        ...,
        [1.28519544e-19, 4.42634338e-19, 5.12413529e-19, ...,
         2.93583588e-24, 2.90901401e-24, 2.93191271e-24],
        [6.53356403e-19, 2.10194468e-18, 2.95195853e-18, ...,
         2.00292115e-23, 2.00182819e-23, 2.00029391e-23],
        [5.71539085e-19, 1.83765927e-18, 2.76924235e-18, ...,
         1.25410509e-23, 1.24847539e-23, 1.24685818e-23]]])
Coordinates:
  * roi_names  (roi_names) <U26 'bankssts L' ... 'transversetemporal R'
  * sub        (sub) <U10 'sub-032412' 'sub-032415' ... 'sub-032469'
  * freqs      (freqs) float64 0.0 1.0 2.0 3.0 4.0 ... 122.0 123.0 124.0 125.0